In [ ]:
import sys
sys.path.append("..")

import random

import time
import math
import random
from io import BytesIO
from pathlib import Path
from typing import Optional, Callable, List, Tuple, Iterable, Generator, Optional, Union

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, IterableDataset
import torchvision.transforms as VT
import torchvision.transforms.functional as VF
from torchvision.utils import make_grid
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.cluster import KMeans

import PIL.Image
import PIL.ImageDraw
import plotly
import plotly.express as px
plotly.io.templates.default = "plotly_dark"
import numpy as np
import pandas as pd

from src.datasets import *
from src.util import *
from src.util.image import * 
from src.algo import Space2d, IFS
from src.datasets import *
from src.models.util import *
from src.models.cnn import *
from src.models.encoder import *
from src.models.decoder import *
from src.models.transform import *
from src.util.embedding import *

def resize(img, scale: float, aa: bool = False):
    return VF.resize(img, [max(1, int(s * scale)) for s in img.shape[-2:]], VF.InterpolationMode.BILINEAR if aa else VF.InterpolationMode.NEAREST, antialias=aa)

In [ ]:
class SymmetricLinear(nn.Module):
    def __init__(
            self,
            in_channels: int,
            out_channels: int,
            activation: Union[None, str, nn.Module] = None,     
            #space_to_depth: bool = False,
            #dropout: float = 0.,
            #batch_norm: bool = False,
    ):
        super().__init__()
        self.activation = activation_to_callable(activation)
        self.linear = nn.Linear(in_channels, out_channels)
        self.bias_out = nn.Parameter(
            torch.randn(in_channels) * self.linear.bias.std()
        )
        
    def forward(self, x, transpose: bool = False):
        if not transpose:
            y = self.linear(x)
            if self.activation:
                y = self.activation(y)
        else:
            y = F.linear(x, self.linear.weight.T, self.bias_out)
            if self.activation:
                y = self.activation(y)
        return y

model = SymmetricLinear(3, 16)
print(f"params: {num_module_parameters(model):,}")
print(model)
print(model(torch.rand(1, 3)).shape)
print(model(torch.rand(1, 16), transpose=True).shape)

In [ ]:
class SymmetricConv2d(nn.Module):
    def __init__(
            self,
            in_channels: int,
            out_channels: int,
            kernel_size: int = 3,
            stride: int = 1,
            groups: int = 1,
            activation: Union[None, str, nn.Module] = "leaky_relu",     
            #space_to_depth: bool = False,
            #dropout: float = 0.,
            #batch_norm: bool = False,
    ):
        super().__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        self.groups = groups
        self.activation = activation_to_callable(activation)
        self.conv_in = nn.Conv2d(in_channels, out_channels, kernel_size, stride)
        self.conv_out = nn.ConvTranspose2d(out_channels, in_channels, kernel_size, stride, output_padding=stride - 1)
        # make weights symmetric
        self.conv_out.weight = self.conv_in.weight
        #self.conv_out.bias = nn.Parameter(torch.randn(in_channels)

    def forward(self, x, transpose: bool = False):
        if not transpose:
            y = self.conv_in(x)
            if self.activation:
                y = self.activation(y)
        else:
            y = self.conv_out(x)
            if self.activation:
                y = self.activation(y)
        return y

model = SymmetricConv2d(3, 16)
print(f"params: {num_module_parameters(model):,}")
print(model)
print(model(torch.rand(1, 3, 10, 10)).shape)
print(model(torch.rand(1, 16, 8, 8), transpose=True).shape)

In [ ]:
class StackedSymmetricAutoencoderConv2d(nn.Module):
    def __init__(
            self,
            shape: Tuple[int, int, int],
            code_size: int,
            kernel_size: Union[int, Iterable[int]] = 3,
            stride: Union[int, Iterable[int]] = 1,
            groups: int = 1,
            channels: Iterable[int] = (16, 32),
            activation: Union[None, str, nn.Module] = "leaky_relu",
            space_to_depth: bool = False,
            dropout: float = 0.,
            batch_norm: bool = False,
    ):
        super().__init__()
        self.shape = tuple(shape)
        self.code_size = code_size
        self.channels = tuple(channels)
        self.kernel_size = kernel_size
        self.stride = stride
        self.groups = groups
        self.activation = activation_to_callable(activation)
        self.layer_index = len(self.channels) + 1
        
        channels = [self.shape[0], *self.channels]
        self.layers = nn.Sequential()
        for idx, ch in enumerate(channels[:-1]):
            ch_in = ch
            ch_out = channels[idx + 1]
            if isinstance(self.kernel_size, int):
                ks = self.kernel_size
            else:
                ks = self.kernel_size[idx]
            if isinstance(self.stride, int):
                stride = self.stride
            else:
                stride = self.stride[idx]

            self.layers.add_module(f"conv_{idx + 1}", SymmetricConv2d(
                ch_in, ch_out, 
                kernel_size=kernel_size, 
                stride=stride, 
                activation=activation,
            ))
            
        self.conv_shapes = []
        with torch.no_grad():
            data = torch.zeros(1, *self.shape)
            for conv in self.layers:
                data = conv(data)
                self.conv_shapes.append(data.shape[-3:])
        # print(self.conv_shapes)

        self.layers.add_module("linear", SymmetricLinear(
            math.prod(self.conv_shapes[-1]), self.code_size,
        ))
        
    def forward(self, x):
        y = self.encode(x)
        # print(f"code: {y.shape}")
        x = self.decode(y)
        return x

    def encode(self, x):
        for idx, layer in zip(range(self.layer_index + 1), self.layers):
            if idx == len(self.layers) - 1:
                x = x.flatten(-3)
            x = layer(x)
            # print("Y", x.shape)
        if self.layer_index < len(self.layers) - 1:
            x = x.flatten(-3)
            
        return x
        
    def decode(self, x):
        if self.layer_index < len(self.conv_shapes) - 1:
            shape = self.conv_shapes[self.layer_index]
            x = x.view(*x.shape[:-1], *shape)
            
        for idx, layer in reversed(list(zip(range(self.layer_index + 1), self.layers))):
            if idx == len(self.conv_shapes) - 1:
                shape = self.conv_shapes[idx]
                x = x.view(*x.shape[:-1], *shape)
            x = layer(x, transpose=True)        
        return x
        
model = StackedSymmetricAutoencoderConv2d(
    (3, 64, 64), 100, 
    channels=(32, ) * 25,
    #channels=(16, 16, 16, 16, 16, 16, 16), 
    #stride=  ( 1,  1,  2,  1,  2,  1,  1),
)
print(f"params: {num_module_parameters(model):,}")
#print(model)
#print("X", model.forward(torch.rand(2, 3, 64, 64)).shape)

for layer_index in range(len(model.layers)):
    model.layer_index = layer_index
    encoded = model.encode(torch.rand(2, 3, 64, 64))
    output = model.decode(encoded)
    print(f"idx={layer_index}, {encoded.shape}, {output.shape}")

#print(output.shape)
display(VF.to_pil_image(output[0]))

In [ ]:
1_000_000 / 47_000

In [ ]:
import base64

In [ ]:
for k in (
    b"tbg-forum@web.de",
    b"forum@tbg2024",
    b"smtp.web.de",
):
    print(base64.b64encode(k))